In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
from colorama import Fore
from colorama import Style
import json
import datetime
from datetime import date, timedelta
d = datetime.datetime.now()

# Automatically set Data Frames to 2 decimal places
pd.options.display.precision = 2

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# print(x.strftime("%d""/""%m""/""%Y"))

# Import API key
from config import nasa_api_key

In [2]:
# Set URL for API
url = "https://api.nasa.gov/neo/rest/v1/feed?"

# URL Format
# https://api.nasa.gov/neo/rest/v1/feed?start_date=START_DATE&end_date=END_DATE&api_key=API_KEY

In [3]:
# Create a Series of dates over the entire year 2019 and call it "days365"
days365=pd.Series(pd.date_range('2019', freq='D', periods=365)).dt.strftime("%Y-%m-%d")
days365

# Create a list of days from the days365 series, to be able to use these in URL calls
day_365_list = []
for days in days365:
    day_365_list.append(str(days))
    
# print(day_365_list)

In [4]:
# Make 1 API calls to NASA to pull JSON files for 01-01-2019 as a test, prior to pull data for the entire year 2019
test_date = "2019-01-01"

print("-----------------------------------")
print("Starting API Call - Retrieving Data")
print("-----------------------------------\n")

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={test_date}&end_date={test_date}&api_key={nasa_api_key}"

print(f"Completed API call for {test_date}\n")

print("-----------------------------------")
print("Data Retrieval Finalised")
print("-----------------------------------")

# Store JSON Response
test_response = requests.get(url).json()

-----------------------------------
Starting API Call - Retrieving Data
-----------------------------------

Completed API call for 2019-01-01

-----------------------------------
Data Retrieval Finalised
-----------------------------------


In [5]:
# response = requests.get(query_url).json()
pprint(test_response)

{'element_count': 10,
 'links': {'next': 'http://www.neowsapp.com/rest/v1/feed?start_date=2019-01-02&end_date=2019-01-02&detailed=false&api_key=LSe3vESzMhcKrbbeo77f9CFvnFj5YgJyxF6WReji',
           'prev': 'http://www.neowsapp.com/rest/v1/feed?start_date=2018-12-31&end_date=2018-12-31&detailed=false&api_key=LSe3vESzMhcKrbbeo77f9CFvnFj5YgJyxF6WReji',
           'self': 'http://www.neowsapp.com/rest/v1/feed?start_date=2019-01-01&end_date=2019-01-01&detailed=false&api_key=LSe3vESzMhcKrbbeo77f9CFvnFj5YgJyxF6WReji'},
 'near_earth_objects': {'2019-01-01': [{'absolute_magnitude_h': 24.2,
                                        'close_approach_data': [{'close_approach_date': '2019-01-01',
                                                                 'close_approach_date_full': '2019-Jan-01 '
                                                                                             '07:58',
                                                                 'epoch_date_close_approach': 154632

In [6]:
# Create lists to hold all of our information we want to store and analyse
asteroid_id=[]
asteroid_name=[]
asteroid_date = []
absolute_magnitude_h=[]
diameter_max_km = []
diameter_min_km = []
is_potentially_hazardous_asteroid=[]
miss_distance_km = []
orbiting_body =[]
relative_velocity =[]
close_approach_date=[]

# Create a counter to be able to change between asteriod dates
x = 0

# Make 365 API calls to NASA to pull JSON files for each date in 2019
print("--------------------------------------------------------------")
print("Starting API Call - Retrieving Data")
print("--------------------------------------------------------------")

for date in day_365_list:
    url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date={date}&api_key={nasa_api_key}"
    
# URL Format - url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2019-01-01&end_date=2019-01-01&api_key={api_key}"

    # Store JSON Response
    response = requests.get(url).json()

    # Create a variable for each date in the JSON Response
    inside = response["near_earth_objects"][date]


# Try and loop through each date the JSON Response and store each element in lists, per asteroid
    try:
        for i in inside:
            
            # Create a variable to be able to loop through each asteriod
            each_asteroid = inside[x]
            
            # Append the asteroid date
            asteroid_date.append(date)
            
            # Append the asteroid ID
            asteroid_id.append(each_asteroid['id'])

            # Append the asteroid name
            asteroid_name.append(each_asteroid['name'])

            # Append the absolute magnitude
            absolute_magnitude_h.append(each_asteroid['absolute_magnitude_h'])

            # Append the max diameter
            diameter_max_km.append(each_asteroid['estimated_diameter']['kilometers']['estimated_diameter_max'])

            # Append the min diameter
            diameter_min_km.append(each_asteroid['estimated_diameter']['kilometers']['estimated_diameter_min'])
            
            # Append "True" or "False", based on whether or not the asteroid was peotentially hazardous
            is_potentially_hazardous_asteroid.append(each_asteroid['is_potentially_hazardous_asteroid'])

            # Append the miss distance 
            miss_distance_km.append(each_asteroid['close_approach_data'][0]['miss_distance']['kilometers'])

            # Append the orbiting body of the asteroid
            orbiting_body.append(each_asteroid['close_approach_data'][0]['orbiting_body'])
            
            # Append the asteroid relative velocity
            relative_velocity.append(each_asteroid['close_approach_data'][0]['relative_velocity']['kilometers_per_second'])
            
            # Append the date the asteroid was considered to be a NEO (Near Earth Object)
            close_approach_date.append(each_asteroid['close_approach_data'][0]["close_approach_date"])
            
            # Move to the next asteroid
            x = x + 1
            
    except:
        print("End of data")
    
    # Reset the asteroid to the first asteroid of the next date
    x = 0   
    
    print(f"Completed API call for {date}, continuing to the next date")
    
print("--------------------------------------------------------------")
print("Data Retrieval Finalised")
print("--------------------------------------------------------------")

--------------------------------------------------------------
Starting API Call - Retrieving Data
--------------------------------------------------------------
Completed API call for 2019-01-01, continuing to the next date
Completed API call for 2019-01-02, continuing to the next date
Completed API call for 2019-01-03, continuing to the next date
Completed API call for 2019-01-04, continuing to the next date
Completed API call for 2019-01-05, continuing to the next date
Completed API call for 2019-01-06, continuing to the next date
Completed API call for 2019-01-07, continuing to the next date
Completed API call for 2019-01-08, continuing to the next date
Completed API call for 2019-01-09, continuing to the next date
Completed API call for 2019-01-10, continuing to the next date
Completed API call for 2019-01-11, continuing to the next date
Completed API call for 2019-01-12, continuing to the next date
Completed API call for 2019-01-13, continuing to the next date
Completed API call 

In [7]:
# Put all data into a data frame
asteroid_df = pd.DataFrame({"Observed Date": asteroid_date, 
                            "Asteroid ID":asteroid_id, 
                            "Asteroid Name": asteroid_name, 
                            "Absolute Magnitude": absolute_magnitude_h, 
                            "Relative Velocity (k/s)" : relative_velocity, 
                            "Close Approach Date": close_approach_date, 
                            "Estimated Max Diameter (km)": diameter_max_km, 
                            "Estimated Min Diameter (km)": diameter_min_km, 
                            "Potentially Hazardous?": is_potentially_hazardous_asteroid, 
                            "Miss Distance (km)": miss_distance_km, 
                            "Orbiting Body": orbiting_body})

# Display head of data frame
asteroid_df.head()

,Observed Date,Asteroid ID,Asteroid Name,Absolute Magnitude,Relative Velocity (k/s),Close Approach Date,Estimated Max Diameter (km),Estimated Min Diameter (km),Potentially Hazardous?,Miss Distance (km),Orbiting Body
0,2019-01-01,3837557,(2019 AN),24.2,8.3685770903,2019-01-01,0.09,0.04,False,19108365.537910876,Earth
1,2019-01-01,3398654,(2007 YS56),25.7,6.4611354444,2019-01-01,0.04,0.02,False,17987605.956517039,Earth
2,2019-01-01,3398652,(2007 YQ56),19.9,15.4294873559,2019-01-01,0.62,0.28,True,11440989.539558679,Earth
3,2019-01-01,3837538,(2018 YV2),24.2,11.7186523264,2019-01-01,0.09,0.04,False,10220722.30201183,Earth
4,2019-01-01,3771017,(2017 EV2),20.8,13.6424741096,2019-01-01,0.41,0.18,False,70335665.49439282,Earth


In [8]:
# Split Observed Date into year, month and day to enable use for further analysis 
asteroid_df["Year"] = pd.DatetimeIndex(asteroid_df["Observed Date"]).year
asteroid_df["Month"] = pd.DatetimeIndex(asteroid_df["Observed Date"]).month
asteroid_df["Day"] = pd.DatetimeIndex(asteroid_df["Observed Date"]).day

# View dataframe with new columns
asteroid_df

,Observed Date,Asteroid ID,Asteroid Name,Absolute Magnitude,Relative Velocity (k/s),Close Approach Date,Estimated Max Diameter (km),Estimated Min Diameter (km),Potentially Hazardous?,Miss Distance (km),Orbiting Body,Year,Month,Day
0,2019-01-01,3837557,(2019 AN),24.20,8.3685770903,2019-01-01,0.09,0.04,False,19108365.537910876,Earth,2019,1,1
1,2019-01-01,3398654,(2007 YS56),25.70,6.4611354444,2019-01-01,0.04,0.02,False,17987605.956517039,Earth,2019,1,1
2,2019-01-01,3398652,(2007 YQ56),19.90,15.4294873559,2019-01-01,0.62,0.28,True,11440989.539558679,Earth,2019,1,1
3,2019-01-01,3837538,(2018 YV2),24.20,11.7186523264,2019-01-01,0.09,0.04,False,10220722.30201183,Earth,2019,1,1
4,2019-01-01,3771017,(2017 EV2),20.80,13.6424741096,2019-01-01,0.41,0.18,False,70335665.49439282,Earth,2019,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5047,2019-12-31,2006239,6239 Minos (1989 QF),18.50,9.0074118926,2019-12-31,1.19,0.53,True,63468291.108281158,Earth,2019,12,31
5048,2019-12-31,3555072,(2011 AH5),26.00,17.9801017354,2019-12-31,0.04,0.02,False,43414047.649301524,Earth,2019,12,31
5049,2019-12-31,3976544,(2020 AA2),27.00,9.714685182,2019-12-31,0.02,0.01,False,1643656.885252799,Earth,2019,12,31
5050,2019-12-31,3893722,(2019 WR4),26.07,4.2368173377,2019-12-31,0.04,0.02,False,4493434.883867377,Earth,2019,12,31


In [9]:
# Export data frame to CSV
asteroid_df.to_csv('Asteroid.csv', index = False)